In [1]:
!git clone https://github.com/Francesco9932/mom_rag

Cloning into 'mom_rag'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 66 (delta 34), reused 17 (delta 4), pack-reused 0
Receiving objects: 100% (66/66), 1.89 MiB | 20.62 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [2]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q -U bitsandbytes
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

!pip install accelerate
!pip install -q pypdf
!pip install -q spacy
!python3 -m spacy download it_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, SpacyTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA

import os

In [4]:
pdf_folder_path = "./mom_rag/bandi/"
documents = []
for file in os.listdir(pdf_folder_path):
    if file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder_path, file)
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())

In [5]:
print(documents[0].page_content)

 
 
 
Data dipubblicazione: 29/08/2023
Nome allegato: Allegato 1-Capitolato Tecnico.pdf
CIG: A004C61876;
Nome procedura: Fornitura disoftware digrafica vettoriale e
software dimodellazione parametrica perilBIMperleesigenze
dell'Area tecnico edilizia.


In [6]:
splitter = SpacyTextSplitter(pipeline = 'it_core_news_sm')
docs = splitter.split_documents(documents)

In [7]:
docs[2]

Document(page_content='2  \n1 PREMESSA  ................................\n\n................................\n\n................................\n\n..............................  \n\n3 \n2 CARATTERISTICHE DELLA FORNITURA  ................................\n\n................................\n\n.................  \n\n4 \n2.1 OGGETTO DELLA FORNITURA  ................................\n\n................................\n\n.............................  4 \n2.2 DURATA DELLA FORNITURA  ................................\n\n................................\n\n...............................  4 \n3 DESCRIZIONE DETTAGLIATA DELLA FORNITURA ................................\n\n................................\n\n..\n\n5 \n3.1 SOTTOSCRIZIONE DI NUOVE LICENZE SOFTWARE AUTODESK  ................................\n\n.........  \n\n5 \n4 CONSEGNA, INSTALLAZIONE E COLLAUDO DEL SOFTWARE  ................................\n\n.................  \n\n6 \n5 LIVELLI DI SERVIZIO E PENALI  .........................

In [8]:
# Define the path to the pre-trained model you want to use
modelPath = "nickprock/sentence-bert-base-italian-uncased"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
# use cpu or gpu
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_kwargs = {'device': device}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/732k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
db = FAISS.from_documents(docs, embeddings)

In [11]:
!nvidia-smi

Wed Apr  3 07:36:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0              41W / 300W |  13078MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
MODEL_NAME = "galatolo/cerbero-7b"

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
# )

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Senza quantizzazione servono 15GB di VRAM

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    # offload_folder="offload"
    # quantization_config=quantization_config
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(
    pipeline=pipeline,
    )


tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/9.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [17]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

system_template = """
Sei un esperto di bandi e concorsi pubblici. Hai memorizzato tutti i bandi mai creati.
Utilizza il contesto nella tua memoria per rispondere alle domande sul bando.
Caratteristiche:
- Tono: formale.
- Non fare utilizzo di bullet points.
Se hai poco contesto rispondi: "Non so aiutarti, scusami"
Se conosci il contesto riformula la risposta invece
----------------
{context}"""

human_template = "{question}"

messages=[
    {
        "role": "system",
        "content": system_template
    },
    {
        "role": "user",
        "content": human_template
    },
]


prompt_template = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt_template)
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 1}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

<s>
Sei un esperto di bandi e concorsi pubblici. Hai memorizzato tutti i bandi mai creati.
Utilizza il contesto nella tua memoria per rispondere alle domande sul bando.
Caratteristiche:
- Tono: formale.
- Non fare utilizzo di bullet points.
Se hai poco contesto rispondi: "Non so aiutarti, scusami"
Se conosci il contesto riformula la risposta invece
----------------
{context}
[|Umano|] {question}
[|Assistente|]


In [13]:
input = "Qual'è l'oggetto della fornitura del bando Frisk?"
result_ = chain(
    input
)
result = result_["result"].strip()

# print(input)
print(result)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>
Sei un esperto di bandi e concorsi pubblici. Hai memorizzato tutti i bandi mai creati.
Utilizza il contesto nella tua memoria per rispondere alle domande sul bando.
Caratteristiche:
- Tono: formale.
- Se utilizzi un elenco puntato al massimo con 3 punti.
Se hai poco contesto rispondi: "Non so aiutarti, scusami"
Se conosci il contesto riformula la risposta invece
----------------
4 2 Caratteristiche Della Fornitura  
2.1 Oggetto Della Fornitura  
Al fine di garantire il continuativo utilizzo della piattaforma  Autodesk , si reputa 
opportuno rinnovare e ampliare  la dotazione  software  in uso attraverso l’acquisto 
in modalità sottoscrizione dei seguenti  prodotti:  
- Architecture Engineering & Construction Collection IC Commercial ; 
- Licenze aggiuntive AutoC AD® full. 


La AEC Collection offre a l CGTE un set di strumenti BIM e CAD supportati da un 
ambiente comune di dati basato sul cloud per realizzare con facilità i progetti , 
dalle fasi iniziali fino alla costruzione.  



In [18]:
input = "Quali sono le caratteristiche principali del software del bando Fornitura di software di grafica vettoriale?"
result_ = chain(
    input
)
result = result_["result"].strip()

print(input)
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Quali sono le caratteristiche principali del software del bando Fornitura di software di grafica vettoriale?
<s>
Sei un esperto di bandi e concorsi pubblici. Hai memorizzato tutti i bandi mai creati.
Utilizza il contesto nella tua memoria per rispondere alle domande sul bando.
Caratteristiche:
- Tono: formale.
- Non fare utilizzo di bullet points.
Se hai poco contesto rispondi: "Non so aiutarti, scusami"
Se conosci il contesto riformula la risposta invece
----------------
4 2 Caratteristiche Della Fornitura  
2.1 Oggetto Della Fornitura  
Al fine di garantire il continuativo utilizzo della piattaforma  Autodesk , si reputa 
opportuno rinnovare e ampliare  la dotazione  software  in uso attraverso l’acquisto 
in modalità sottoscrizione dei seguenti  prodotti:  
- Architecture Engineering & Construction Collection IC Commercial ; 
- Licenze aggiuntive AutoC AD® full. 


La AEC Collection offre a l CGTE un set di strumenti BIM e CAD supportati da un 
ambiente comune di dati basato sul clo

In [ ]:
input = "Devo creare un nuovo bando di Sofware CRM Salesforce, mi generi il capitolo caratteristiche della fornitura. Fornisci in output un discorso per spiegarlo"
result_ = chain(
    input
)
result = result_["result"].strip()
# print(input)
print(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>
Sei un esperto di bandi e concorsi pubblici. Hai memorizzato tutti i bandi mai creati.
Utilizza il contesto nella tua memoria per rispondere alle domande sul bando.
Caratteristiche:
- Tono: formale.
- Se utilizzi un elenco puntato al massimo con 3 punti.
Se hai poco contesto rispondi: "Non so aiutarti, scusami"
Se conosci il contesto riformula la risposta invece
----------------
4 
 
Direzione centrale Tecnologia, Informatica e Innovazione  
 
1.

Preparazione e consegna di un programma di progetto.

 


2. Schedulazione e facilitazione dei workshop di progetto richiesti.  


3. Definizione di un elenco dei pre -requisiti per garantire il successo 
dell'implementazione.  


4.

Installazione On -Premise in due ( 2) ambienti.  


5.

Installazione fino a tre (3) Remote ETL Engine Server  
6.

Preparazione e consegna di un documento di c ostruzione.  


7. Configurazione di base di dieci (10) utenti e tre (3) ruoli  
8.

Configurazione di un ETL standard del prodotto per CMDB BMC  
9